In [319]:
import torch

In [320]:
t1 = torch.tensor(4.)
t1

tensor(4.)

In [321]:
t2 = torch.tensor((4,6,8))
t2

tensor([4, 6, 8])

In [322]:
t1 == 4.

tensor(True)

In [323]:
matrix = torch.tensor([[1.,2.,3.],[2.,4.,6.],[3.,5.,7.]])
r1 = matrix[0]
r2 = matrix[1]
r3 = matrix[2]

In [324]:
print("\tMatrix\n",matrix)

	Matrix
 tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 5., 7.]])


In [325]:
r2 = r2 - (2 * r1)
matrix[1] = r2
matrix

tensor([[1., 2., 3.],
        [0., 0., 0.],
        [3., 5., 7.]])

In [326]:
r3 = -(r3 - (3 * r1))
matrix[2] = r3
matrix

tensor([[1., 2., 3.],
        [0., 0., 0.],
        [-0., 1., 2.]])

In [327]:
matrix[2][0] = 0

In [328]:
matrix

tensor([[1., 2., 3.],
        [0., 0., 0.],
        [0., 1., 2.]])

## **LINEAR REGRESSION**

In [329]:
data = torch.tensor([[1180],[2000],[1500],[1200],[1860],[3000],[3500],[2200],[2650],[1000]],dtype=torch.float32)
target = torch.tensor([200000,500000,350000,280000,370000,650000,700000,520000,560000,175000],dtype=torch.float32)

In [330]:
w = torch.randn(1, 1 ,requires_grad=True)
b = torch.randn(1, requires_grad=True)

In [331]:
model = lambda x : x @ w.t() + b

In [332]:
preds = model(data).squeeze()
preds

tensor([ 644.6548, 1092.3519,  819.3659,  655.5743, 1015.9158, 1638.3240,
        1911.3101, 1201.5464, 1447.2338,  546.3799],
       grad_fn=<SqueezeBackward0>)

In [333]:
def mse(y_true,y_pred):
  diff = y_pred - y_true
  return torch.sum(diff ** 2) / diff.numel()

In [334]:
difference = (preds - target)
difference.shape

torch.Size([10])

In [335]:
loss = mse(target,preds)
loss

tensor(2.1440e+11, grad_fn=<DivBackward0>)

In [336]:
loss.backward()

In [337]:
print(w)
print(w.grad)
print(b)
print(b.grad)

tensor([[0.5460]], requires_grad=True)
tensor([[-1.9925e+09]])
tensor([0.4078], requires_grad=True)
tensor([-858805.4375])


In [338]:
# lr = 1e-5
# with torch.no_grad():
#   w -= w.grad * lr
#   b -= b.grad * lr
#   w.grad.zero_()
#   b.grad.zero_()

In [339]:
model(data)

tensor([[ 644.6548],
        [1092.3519],
        [ 819.3659],
        [ 655.5743],
        [1015.9158],
        [1638.3240],
        [1911.3101],
        [1201.5464],
        [1447.2338],
        [ 546.3799]], grad_fn=<AddBackward0>)

In [340]:
epochs = 100
lr = 1e-7
for i in range(epochs):
  preds = model(data)
  loss = mse(preds,target)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * lr
    b -= b.grad * lr
    w.grad.zero_()
    b.grad.zero_()

In [341]:
preds = model(data)
loss = mse(preds,target)

In [342]:
loss

tensor(5.5163e+10, grad_fn=<DivBackward0>)

In [343]:
preds

tensor([[218747.5625],
        [370757.4375],
        [278068.4688],
        [222455.1094],
        [344804.5312],
        [556135.2500],
        [648824.1875],
        [407833.0000],
        [491253.0625],
        [185379.5469]], grad_fn=<AddBackward0>)

In [344]:
target

tensor([200000., 500000., 350000., 280000., 370000., 650000., 700000., 520000.,
        560000., 175000.])

In [345]:
import pandas as pd
df = pd.read_csv("/content/sample_data/california_housing_train.csv")
df = df[['total_rooms','total_bedrooms','housing_median_age','median_house_value']]

In [346]:
X = df.drop('median_house_value',axis=1).to_numpy()[:100]
y = df['median_house_value'].to_numpy()[:100]

In [347]:
X = torch.from_numpy(X).to(torch.float32)
y = torch.from_numpy(y).to(torch.float32)

In [348]:
from torch.utils.data import TensorDataset
ds = TensorDataset(X,y)

In [349]:
from torch.utils.data import DataLoader
batch_size = 10
dataload = DataLoader(ds,batch_size,shuffle=True)

In [350]:
for i,j in dataload:
  print(i,j)
  break

tensor([[ 805.,  143.,    5.],
        [2652.,  606.,    5.],
        [1039.,  256.,   17.],
        [1995.,  523.,   32.],
        [1678.,  322.,   28.],
        [2276.,  594.,   16.],
        [1291.,  248.,   48.],
        [1327.,  262.,   38.],
        [4789., 1175.,   34.],
        [2578.,  634.,   25.]]) tensor([96300., 84300., 63500., 43300., 94900., 93800., 48600., 60800., 58400.,
        62200.])


In [351]:

list(model.parameters())

[Parameter containing:
 tensor([[0.3463, 0.2517, 0.1865]], requires_grad=True),
 Parameter containing:
 tensor([0.4686], requires_grad=True)]

In [352]:
preds = model(X)

loss_fn = torch.nn.functional.mse_loss

In [371]:
def fit(epochs,lr,model,train_dl):
  model = torch.nn.Linear(3,1)

  for epoch in range(epochs):
    for xt,yt in train_dl:
      pred = model(xt)
      loss_fn = torch.nn.functional.mse_loss
      loss = loss_fn(pred,yt)

      loss.backward()


      opt = torch.optim.SGD(model.parameters(),lr=lr)
      opt.step()

      opt.zero_grad()

    if (epoch % 10 == 0):
      print(("Epoch:{}||Loss:{}").format(epoch,loss))
  return model

In [377]:
epochs = 100
learning_rate=1e-7
model = fit(epochs,learning_rate,model,dataload)

<ipython-input-371-8ea5e07e8606>:8: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(pred,yt)


Epoch:0||Loss:3297350656.0
Epoch:10||Loss:1567410688.0
Epoch:20||Loss:1854821504.0
Epoch:30||Loss:2796915712.0
Epoch:40||Loss:1416071040.0
Epoch:50||Loss:539750720.0
Epoch:60||Loss:3882665216.0
Epoch:70||Loss:2960319488.0
Epoch:80||Loss:2232125184.0
Epoch:90||Loss:529818144.0


In [378]:
for i in zip(model(X),target):
  print(i)

(tensor([204589.3281], grad_fn=<UnbindBackward0>), tensor(200000.))
(tensor([270110.5000], grad_fn=<UnbindBackward0>), tensor(500000.))
(tensor([27214.9180], grad_fn=<UnbindBackward0>), tensor(350000.))
(tensor([56063.7266], grad_fn=<UnbindBackward0>), tensor(280000.))
(tensor([54972.8047], grad_fn=<UnbindBackward0>), tensor(370000.))
(tensor([57679.5781], grad_fn=<UnbindBackward0>), tensor(650000.))
(tensor([106814.3906], grad_fn=<UnbindBackward0>), tensor(700000.))
(tensor([34460.5547], grad_fn=<UnbindBackward0>), tensor(520000.))
(tensor([172142.5000], grad_fn=<UnbindBackward0>), tensor(560000.))
(tensor([60632.1836], grad_fn=<UnbindBackward0>), tensor(175000.))
